In [1]:
import pandas as pd

In [2]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
TEAM_NUMBER = "TeamDemo"

# File with AOIS definitions 
AOIS_DEFINITION_FILE = "../raw-data/"+TEAM_NUMBER+"/aois definitions/aois.csv"

# This file should be automatically generated from "4. Mapping Fixations and Saccades to AOIs"
FIFXATIONS_SACCADES_WITH_AOIS_FILE = "data/fixationAndSaccadesDataWithAOIs.csv"

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 1. Identify Dwells
# ("a dwell is defined as one visit in an AOI, from entry to exit" Holmqvist et al. 2010, page 262 - See book reference in lecture slides)
# 
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read aois definition
aois_df = pd.read_csv(AOIS_DEFINITION_FILE)

In [ ]:
# Display aois definition
display(aois_df)

In [ ]:
# get list of AOIs
aois = aois_df["AOI"].tolist()
print(f'AOIs: {aois}')

In [ ]:
# Read fixation and saccade data with AOIs using pandas library
data = pd.read_csv(FIFXATIONS_SACCADES_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview fixation and saccade data with aois
data.head()

In [ ]:
# Derive dwells

In [ ]:
dwells = pd.DataFrame()

# Iterate through the different aois
for aoi in aois:
    
    #  Group by consecutive same value (CSAV) and Respondent (i.e., participant)
    # documentation for the aprroach used to group columns by consecutive same value https://towardsdatascience.com/pandas-dataframe-group-by-consecutive-same-values-128913875dba
    csavGroups = data.groupby([(data[aoi].shift() != data[aoi]).cumsum(),'Respondent','SourceStimuliName'])
    
    # Keep only groups where the AOI is activited (aoiName==1) 
    csavGroups = {key: value for key, value in csavGroups if value[aoi].unique()==[1]}
    
    # compute a set of metrics for each dwell (i.e., a group where the AOI is activiated)     
    for key in csavGroups:
        group = csavGroups[key]
                
        dwells = pd.concat([dwells, pd.DataFrame.from_records([{
            # Participant (i.e., Respondent)
            'Respondent': group.iloc[0]['Respondent'], #The respondent should be the same within the whole group
            # Participant (i.e., Respondent)
            'SourceStimuliName': group.iloc[0]['SourceStimuliName'], #The SourceStimuliName should be the same within the whole group
            # The aoi visited during the dwell
            'VisitedAOI': aoi,
            # Timestamp when the aoi was entered
            'Dwell Start': group.iloc[0]['Fixation Start'] if not pd.isnull(group.iloc[0]['Fixation Start']) else group.iloc[0]['Saccade Start'],
            # Timestamp when the aoi was exited
            'Dwell End' : group.iloc[-1]['Fixation End'] if not pd.isnull(group.iloc[-1]['Fixation End']) else group.iloc[-1]['Saccade End'] ,
            # Number of Fixations in Dwell (Numerosity measure, cf. documentation/ETmetrics.csv)
            'Number of Fixations in Dwell': group[ (group['FixID'].notnull()) ].shape[0],
            # Number of Saccades in Dwell (Numerosity measure, cf. documentation/ETmetrics.csv)
            'Number of Saccades in Dwell': group[ (group['SacID'].notnull()) ].shape[0]
        }])])

In [ ]:
# Dwell time (Position measure, cf. documentation/ETmetrics.csv)
dwells['Dwell Time'] = dwells['Dwell End']-dwells['Dwell Start'] 

In [ ]:
#Sort dwells by 'Respondent', Dwell Start' in an ascending order
dwells = dwells.sort_values(by=['Respondent','Dwell Start'])

#Reset index
dwells = dwells.reset_index(drop=True)

In [ ]:
#Preview dwells
dwells.head()

In [ ]:
# export dwells to csv
dwells.to_csv("data/dwells.csv",  index=False)